#### Title: 02_qc_matched_data
#### Purpose: Map updated nodes labels for food tree based on matches from step 01. Update L2 categories for ingredietns
#### Date: March 28, 2024
#### Author: Jules Larke

In [2]:
import pandas as pd
import numpy as np
import string

In [3]:
taxa = pd.read_csv('../../data/01/updated_taxonomy_080323.csv', usecols=['Ingredient code', 'Ingredient description', 'Level.code', 'correct_code']) # manual matching from step 01
labels = pd.read_csv('../../data/02/NodeLabelsMCT_updated.txt', sep='\t') # updated food tree node labels for ingredients
ingredients = pd.read_csv('../../data/00/wweia_crp_unique_ingred_codes.csv') # list of all unique ingredient codes in NHANES dataset

In [4]:
labels['Level.code'] = labels['Level.code'].astype(int)

In [5]:
ingredients = ingredients[['ingred_code']]

In [6]:
taxa['Level.code'] = np.where(~taxa['correct_code'].isnull(),taxa['correct_code'],taxa['Level.code'])

In [7]:
punct = string.punctuation

In [8]:
def clean_text(text):
    text = "".join([word for word in text if word not in punct])
    return text

taxa['Ingredient description'] = taxa['Ingredient description'].apply(lambda x: clean_text(x))
taxa['Ingredient description'] = taxa['Ingredient description'].str.replace(' ', '_')

In [9]:
taxa = taxa[['Level.code', 'Ingredient description', 'Ingredient code']]

In [10]:
taxa['Level.code'] = taxa['Level.code'].astype('int64')

In [11]:
taxa = taxa[taxa['Ingredient code'].isin(ingredients['ingred_code'])]

In [12]:
taxa = taxa.rename(columns={'Ingredient description': 'Main.food.description', 'Level.code': 'FoodID'})

In [13]:
labels

,Level.code,Main.food.description
0,1,Milk_and_Milk_Products
1,11,Milks_and_milk_drinks
2,110,Milk_human
3,111,Milk_cow
4,1111,Milk_fluid_whole
...,...,...
1787,9533,Fluid_replacements
1788,9534,Other_functional_beverages
1789,954,Meal_replacement
1790,99,Added_vitamins


In [14]:
to_replace = {'Milk_desserts_sauces_gravies':'Milk_desserts',
'Frozen_and_shelfstable_plate_meals_soups_and_gravies':'Broths_and_stocks',
'Egg_mixtures':'Eggs',
'Nuts_nut_butters_and_nut_mixtures':'Nuts_and_nut_butters',
'Seeds_and_seed_mixtures':'Seeds',
'Quick_breads':'Flour_and_dry_mixes',
'Cakes_cookies_pies_pastries_bars':'Bars',
'Crackers_and_salty_snacks_from_grain':'Salty_snacks_from_grain_products',
'Citrus_fruits_juices':'Citrus_fruits_and_juices',
'Tomatoes_and_tomato_mixtures':'Tomatoes_and_tomato_sauces',
'Formulated_nutrition_beverages_energy_drinks_sports_drinks_function':'Formulated_nutrition_beverages_energy_drinks_sports_drinks'}

In [15]:
labels['Main.food.description'] = labels['Main.food.description'].replace(to_replace)

In [16]:
x = labels

In [17]:
x.rename(columns={'Level.code':'FoodID'},inplace=True)

In [18]:
y = taxa.merge(x, on='FoodID', how='left')

In [20]:
# full mapping of taxonomy to ingredients
y.rename(columns={'Main.food.description_x':'ingred_desc', 'Main.food.description_y':'taxon'},inplace=True)
y.to_csv('../../data/02/all_ingred_codes_taxa_082323.csv',index=None)

In [21]:
# list of all unique ingredients in wweia data
taxa[['Main.food.description', 'Ingredient code']].to_csv('../../data/02/wweia_ingredient_list.csv', index=None)

In [22]:
# Count the number of ingredient codes within each of the distinct taxonomic levels.

In [23]:
taxa_count = taxa.FoodID.value_counts().to_frame()

In [24]:
taxa_count.reset_index(inplace=True)

In [25]:
taxa_count.rename(columns={'index':'Level.code', 'FoodID':'count'}, inplace=True)

In [26]:
taxa.sort_values(by='Ingredient code')

,FoodID,Main.food.description,Ingredient code
0,81101,Butter_salted,1001
1,81107,Butter_whipped_with_salt,1002
2,81101,Butter_oil_anhydrous,1003
3,14101,Cheese_blue,1004
4,14102,Cheese_brick,1005
...,...,...,...
2749,2522,Sausage_pork_chorizo_link_or_ground_cooked_pan...,100173
2727,25230,REDUCED_SODIUM_Ham_sliced_prepackaged_deli_mea...,907028
2731,25230,REDUCED_SODIUM_Turkey_breast_sliced_prepackaged,907081
2732,25230,REDUCED_SODIUM_Chicken_breast_deli_rotisserie_...,907961


In [27]:
labels.rename(columns={'FoodID':'Level.code'}, inplace=True)

In [28]:
# count of all foods per taxon
taxa_count.merge(labels, on='Level.code', how='left').to_csv('../../data/02/taxa_level_count.csv', index=None)

In [29]:
taxa.to_csv('../../data/02/wweia_ingredient_taxa.txt', sep='\t', index=None)

In [31]:
labels.to_csv('../../data/02/NodeLabelsMCT_updated_082323.txt', sep='\t', index=None)